In [44]:
from enum import Enum
from typing import List
from collections import defaultdict, deque

import sys

sys.path.append('..')
from utils.utils import fmt

class Edge(Enum):
    OPEN = ' '
    WALL = 'x'

class Board:
    def __init__(self, grid: List[List[str]]):
        self.grid = grid
        self.M = 8
        self.N = 8
    
    def __str__(self):
        output = []
        for i, row in enumerate(self.grid):
            line = []
            for j, cell in enumerate(row):
                line.append(cell)
            output.append(line)

        M, N = len(output), len(output[0])

        for i, line in enumerate(output):
            for j, cell in enumerate(line):
                if cell == ' ':
                    num_adj_os = 0
                    for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < M and 0 <= nj < N:
                            if output[ni][nj] == 'o':
                                num_adj_os += 1
                    if num_adj_os > 1:
                        output[i][j] = 'o'
                elif cell == '.':
                    num_adj_xs = 0
                    for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < M and 0 <= nj < N:
                            if output[ni][nj] == 'x':
                                num_adj_xs += 1
                    if num_adj_xs >= 1:
                        output[i][j] = 'x'

        for i, line in enumerate(output):
            for j, cell in enumerate(line):
                if cell == 'x':
                    output[i][j] = fmt(' ', 1, True)
                elif cell == 'o':
                    output[i][j] = fmt(' ', 2, True)
                elif '0' <= cell <= 'Z':
                    output[i][j] = fmt(cell, 4, True, True)
                elif cell == '.':
                    output[i][j] = '┼'
                elif cell == '|':
                    output[i][j] = '│'
                elif cell in ('-'):
                    output[i][j] = '─'
                # else:
                #     output[i][j] = fmt(cell, 236, True)

        for i, line in enumerate(output):
            for j, cell in enumerate(line):
                    if j % 2 == 1:
                        # output[i][j] = f" {cell}{cell} "
                        output[i][j] = f" {cell} "
                        # output[i][j] = f"{cell}"
        
        # Shave top and bottom edges
        for j in range(N):
            if j % 2 == 0:
                output[0][j] = '┬'
                output[-1][j] = '┴'

        # Shave left and right edges
        for i in range(M):
            if i % 2 == 0:
                output[i][0] = '├'
                output[i][-1] = '┤'
        
        # Shave corners
        output[0][0] = '┌'
        output[0][-1] = '┐'
        output[-1][0] = '└'
        output[-1][-1] = '┘'
                
        # result = '\n'.join([''.join(line[1:-1]) for line in output[1:-1]])
        result = '\n'.join([''.join(line) for line in output])
        return result

def parse_text_file(file_path):
    inputs = [list(line.strip()) for line in open(file_path, 'r').readlines()]

    M, N = len(inputs) // 2, len(inputs[0]) // 2
    print(M, N)

    dots = list()
    walls = defaultdict(set)
    path_edges = defaultdict(set)
    path_nodes = set()

    for i, line in enumerate(inputs):
        for j, char in enumerate(line):
            if '0' <= char <= '9' or 'A' <= char <= 'Z':
                r, c = i // 2, j // 2
                dots.append((char, (r, c)))
            if char == 'x':
                r2, c2 = i // 2, j // 2
                # Horizontal wall
                if i % 2 == 0:
                    r1, c1 = r2 - 1, c2
                # Vertical wall
                else:
                    r1, c1 = r2, c2 - 1
                walls[(r1, c1)].add((r2, c2))
                walls[(r2, c2)].add((r1, c1))
            if char == 'o':
                r2, c2 = i // 2, j // 2
                # Horizontal edge
                if i % 2 == 0:
                    r1, c1 = r2 - 1, c2
                # Vertical edge
                else:
                    r1, c1 = r2, c2 - 1
                path_edges[(r1, c1)].add((r2, c2))
                path_edges[(r2, c2)].add((r1, c1))
                path_nodes.add((r1, c1))
                path_nodes.add((r2, c2))

    dots.sort()
    print(dots)
    print(walls)
    print(path_edges)

    start_pos = dots[0][1]
    path = [start_pos]
    queue = deque([start_pos])
    visited = {start_pos}

    while queue:
        node = queue.popleft()
        neighbors = path_edges[node]
        assert len(neighbors) <= 2
        for neighbor in neighbors:
            if neighbor not in visited:
                queue.append(neighbor)
                path.append(neighbor)
                visited.add(neighbor)
                
    assert path_nodes == visited
    print(path)

    return Board(inputs)

file_path = 'text_inputs/2025-05-04.txt'
board = parse_text_file(file_path)
print(board)

8 8
[('1', (0, 1)), ('2', (5, 5)), ('3', (6, 4)), ('4', (2, 5)), ('5', (4, 6)), ('6', (7, 6)), ('7', (5, 2)), ('8', (3, 1)), ('9', (1, 3)), ('A', (2, 2))]
defaultdict(<class 'set'>, {(1, 6): {(1, 7), (2, 6)}, (1, 7): {(1, 6)}, (1, 5): {(2, 5)}, (2, 5): {(2, 4), (1, 5)}, (2, 6): {(1, 6)}, (2, 4): {(2, 5)}, (3, 4): {(4, 4), (3, 5)}, (3, 5): {(3, 4)}, (3, 3): {(4, 3)}, (4, 3): {(3, 3), (4, 2)}, (4, 4): {(3, 4)}, (4, 2): {(4, 3)}, (5, 2): {(5, 3), (6, 2)}, (5, 3): {(5, 2)}, (5, 1): {(6, 1)}, (6, 1): {(5, 1), (6, 0)}, (6, 2): {(5, 2)}, (6, 0): {(6, 1)}})
defaultdict(<class 'set'>, {(0, 1): {(0, 2)}, (0, 2): {(0, 1), (0, 3)}, (0, 3): {(0, 2), (0, 4)}, (0, 4): {(0, 3), (0, 5)}, (0, 5): {(0, 4), (0, 6)}, (0, 6): {(0, 7), (0, 5)}, (0, 7): {(1, 7), (0, 6)}, (1, 7): {(0, 7), (2, 7)}, (2, 7): {(3, 7), (1, 7)}, (3, 7): {(4, 7), (2, 7)}, (4, 7): {(3, 7), (5, 7)}, (5, 7): {(5, 6), (4, 7)}, (5, 5): {(5, 6)}, (5, 6): {(5, 5), (5, 7)}})
[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 7), (2